# Introduction
KNN is a supervised machine learning model, meaning the algorithm relies on learning from labeled data to predict a given output. KNN is a classification problem. KNN computes the distances of all data points. The distances are sorted, and the first "k" neighbors determines the what class the data point of interest is assigned to.

For this example, we are looking at creditcard.csv to predict if a transaction was fradulent or not. There are 30 features and class labels. We will construct two KNN models, one with SKlearn and a homegrown one. The results of each one are shown below. Happy learning!

# KNN with SKLearn

In [2]:
# Load Libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
# Read in dataframe
credit_cards = pd.read_csv("creditcard.csv")
credit_cards.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Here we see that there are 30 columns of different features and column 31 represents the class label (y). We want to separate features (X) from the labels (y). We will use the 30 features to predict the associated class of each instance. Below shows how you can use pandas indexing to accomplish this separation.

The next step is to split the data into four parts: X_train and x_test, and Y_train and y_test. To accomplish this, we will use SKlearn's train_test_split method. For this case, we are allocating 25% of our data for testing and 75% for training. Then, we will normalize the data using the MinMaxScalar to standardize the data. 

In [17]:
# Split up data from features and class labels
X = credit_cards.iloc[:,1:30]
y = credit_cards["Class"]

# Split up training and testing data
subsample_rate = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
X_train, _, y_train, _ = train_test_split(X_train, y_train, stratify=y_train, train_size=subsample_rate, random_state=42)
X_test, _, y_test, _ = train_test_split(X_test, y_test, stratify=y_test, train_size=subsample_rate, random_state=42)

# Standardize data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Now we have a lot to consider. How many neighbors (k) will optimize our predictions? Should we use uniform or weighted distance to calculate knn? To answer this, we can use tune hyperparameters using GridSearch. The first step is to set up a dictionary of parameters you want to test out. Here we are trying to select the best combination from three parameters: k, scoring metric, and uniform or weighted distance. GridSearch will iterate over these options to determine the best set of parameters for the model.

After the parameters are set-up the next step is set up a pipeline, the purpose of a pipeline is to assemble several steps that can be cross-validated together while setting different parameters (what we did in the previous step). Here, it standardizes the final estimator. We put the pipeline and parameters in the gridsearch and determine the best set of parameters.

In [18]:
# Initialize Classifier
knn = KNeighborsClassifier()

# Set up parameters
parameters = {
    # Number of neighbors
    'knn__n_neighbors': list(range(1,11)),
    # Numeric value denotes a scoring metric
    'knn__p': list(range(1,4)),
    # Uniform or weighted distance
    'knn__weights': ['uniform', 'distance']
}

# Set up pipeline
pipe = Pipeline([
    ('standardize', StandardScaler()),
    ('knn',knn)
])

# Set up Gridsearch
knn_gs = GridSearchCV(pipe,parameters,cv=10,n_jobs=-1,scoring="accuracy")

In [19]:
knn_gs

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('standardize',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                                             metric_params=None,
                                                             n_jobs=None,
                                                             n_neighbors=5, p=2,
                                                             weights='uniform'))],
                                verbo

Now, with our new gridsearch, we will fit our training today to find the best hyperparameter combination and display the results below.

In [20]:
# Conduct parameter search, find best estimator
knn_gs.fit(X_train, y_train)
knn_gs.best_estimator_

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=8, p=1,
                                      weights='distance'))],
         verbose=False)

In [ ]:
# Make predictions
y_true, y_pred = y_test, knn_gs.predict(X_test)

Now we will make predictions and store the results in a dataframe.

In [167]:
# Capture results in dataframe
results = pd.DataFrame(columns=["Model","K (Neighbors)","P (Scoring Metric)", "Weights", 
                                "Training Accuracy Score","Testing Accuracy Score"])
results.loc[len(results)] = ["KNN Classifier", knn_gs.best_params_['knn__n_neighbors'],knn_gs.best_params_['knn__p'],
                             knn_gs.best_params_['knn__weights'], knn_gs.best_score_ ,accuracy_score(y_true, y_pred)]
results

,Model,K (Neighbors),P (Scoring Metric),Weights,Training Accuracy Score,Testing Accuracy Score
0,KNN Classifier,8,1,distance,0.999605,0.998771


In [63]:
# Show results
prediction_results = pd.DataFrame([y_pred,y_true]).T
prediction_results.columns = ["Y-pred","Y-true"]
prediction_results

,Y-pred,Y-true
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
5691,0,0
5692,0,0
5693,0,0
5694,0,0


After standardizing data and conducting hyperparameter tuning, we achieved a high accuracy score for both our training and testing data. We performed slightly better with the training set, but we scored over 9

# Homegrown KNN

Can we achieve similar results without using SKlearn? To test this out, we will construct KNN from scratch (homegrown) in an object oriented style. The parameters include the "k", the scoring function, and whether we will use distance or uniform weights. We have to re-load and restandardize our data from the previous step.

In [161]:
class KNN(object):

    def __init__(self, k_neighbors=8, p=1,d_type="weighted"):
        self.k_neighbors=k_neighbors
        self.p = p
        self.training_data = np.array([])
        self.training_labels = np.array([])
        self.d_type=d_type
        
    def distance(self,x1,x2,d_type):
        if d_type == "uniform":
            return np.linalg.norm(x1-x2, ord=self.p, axis=1)
        else:
            distances = 1/np.linalg.norm(x1-x2, ord=self.p, axis=1)
            distances[distances > 1e308] = 0
            return distances
    
    def accuracy(self,y_true,y_pred):
        return (y_pred == y_true).mean()
    
    def fit(self,X,y):
        self.training_data = X
        self.training_labels = y
        return self
    
    def nearest_neighbors(self, X):
        nearest_indices = np.zeros(shape=(X.shape[0], 
                                          self.k_neighbors), dtype=np.int)-1
        nearest_distances = np.zeros(shape=(X.shape[0], 
                                            self.k_neighbors), dtype=np.int)-1 
        
        for i in range(len(X)):
            distances = self.distance(X[i], self.training_data,"distance")
            indexes = np.argsort(distances)[:self.k_neighbors]
            nearest_indices[i] = indexes
            nearest_distances[i] = distances[indexes]
            
        return (nearest_indices, nearest_distances)
    
    def predict(self, X):
        y = np.zeros(shape=(X.shape[0],))
        indices_distances = self.nearest_neighbors(X)
        y_labels = self.training_labels[indices_distances[0]]
        distances = indices_distances[1]
        
        if self.d_type == "uniform":
            for i in range(X.shape[0]):
                y[i] = np.argmax(np.bincount((y_labels[i])))
            return y
        else:
            for i in range(X.shape[0]):
                counts = np.bincount(y_labels[i])
                weighted_sum = counts[y_labels[i]] * distances[i]
                y[i] = y_labels[i][np.argmax(weighted_sum)]
            return y
    
    def score(self, X, y):
        y_pred = self.predict(X)
        acc = self.accuracy(y, y_pred)
        loss = np.linalg.norm(y - y_pred, ord=self.p)
        scores_count = {"acc":acc, "loss":loss}
        return scores_count

In [ ]:
# Set best hyperparameters
np.random.seed(42) 

# Reload data
subsample_rate = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)
X_train, _, y_train, _ = train_test_split(X_train, y_train, stratify=y_train, train_size=subsample_rate, random_state=42)
X_test, _, y_test, _ = train_test_split(X_test, y_test, stratify=y_test, train_size=subsample_rate, random_state=42)

# Standardize data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Use Homegrown knn
knn_scratch = KNN(k_neighbors=8, p=1, d_type='weighted')   
knn_scratch.fit(X_train, y_train.to_numpy()) 
train_score = knn_scratch.score(X_train,y_train)
test_score = knn_scratch.score(X_test,y_test.to_numpy())
y_pred_train = knn_scratch.predict(X_train)
trainAcc = knn_scratch.score(y_train.to_numpy(), y_pred_train)
y_pred_test = knn_scratch.predict(X_test)
testAcc = knn_scratch.score(y_test.to_numpy(), y_pred_test)

In [165]:
trainAcc = knn_scratch.score(y_train.to_numpy(), y_pred_train)
y_pred_test = knn_scratch.predict(X_test)
testAcc = knn_scratch.score(y_test.to_numpy(), y_pred_test)

In [168]:
results.loc[len(results)] = ["KNN Classifier Homegrown", knn_gs.best_params_['knn__n_neighbors'],knn_gs.best_params_['knn__p'],
                             knn_gs.best_params_['knn__weights'],trainAcc["acc"],testAcc["acc"]]
results

,Model,K (Neighbors),P (Scoring Metric),Weights,Training Accuracy Score,Testing Accuracy Score
0,KNN Classifier,8,1,distance,0.999605,0.998771
1,KNN Classifier Homegrown,8,1,distance,1.000000,0.998244


It looks like our homegrown KNN performed better on the training data compared to the KNN Classifer with SkLearn and the testing data from KNN Homegrown. The testing data from KNN Homegrown performed slightly worses compared to SKLearn, but it is pretty close! This shows you can build comparable models from scratch rather than using libraries (if you so choose to!).